In [3]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from linecache import getline as gl
import pandas as pd
from scipy.signal import convolve2d
from itertools import product
import os
import pickle
import math
import pdb
import cartopy.crs as ccrs
import cartopy.feature as cfeature


%matplotlib

RADIUS = 6371
airport_list = ['JFK', 'EWR', 'LGA',
                'PUQ', 
                'FRA',
                'TLV', 'SDV', 
                'WUH',
                'DEL', 
                'CDG', 'ORY',
                'LTN', 'LHR', 'LGW',
                'TXL', 'SXF',
                'SFO', 'SJC', 'OAK',
                'MIA', 'FLL'
               ]

def deg_dist_at_lat(lat): ## Latitude in degrees!
    assert lat >= -90 and lat <= 90
    return math.cos(lat * 2 * math.pi / 360) * RADIUS * 2 * math.pi / 360

# Test, compare to https://en.wikipedia.org/wiki/Longitude#Length_of_a_degree_of_longitude
#_ = [print(f'1 degree at latitutde {int(lat)} = {deg_dist_at_lat(lat)} km') for lat in(range(-90,105,15))]   

Using matplotlib backend: Qt5Agg


In [4]:
def process(n):
    line = gl(filename, n).replace('\n','').split(' ')
    return line[0], line[-1]

filename = 'data/gpw_v4_population_density_rev11_2020_2pt5_min.asc'
data = np.loadtxt(filename, skiprows=6)
data[(data==-9999)] = 0

specs = {k:float(v) for k,v in map(process,range(1,7))}
specs['data'] = data

In [5]:
## Coordinate format: Latitude N , Longitude E.
## Longitude / East / x-axis / columns / (-180,180)
## latitude / North / y-axis / rows / (-90,90).

def get_coordinate_functions(specs):
    cellsize = specs['cellsize']
    xll = specs['xllcorner']
    yll = specs['yllcorner']
    nrows = specs['nrows']
    ncols = specs['ncols']

    def longitude2idx(long):
        long = np.array(long)
        #assert np.all(long >= xll)
        return ((long - xll)/cellsize).astype(int)
    def latitude2idx(lat):
        lat = np.array(lat)
        #assert np.all(lat >= yll)
        return (nrows - (lat - yll)/cellsize).astype(int)
    specs['long_f'] = longitude2idx
    specs['lat_f'] =  latitude2idx
    return specs

def row2lat(row, specs):
    nrows = specs['nrows']
    yll = specs['yllcorner']
    cellsize = specs['cellsize']
    return (nrows - row) * cellsize + yll


def augment(long, lat, specs, scan=30, avg=15): #resolution in degrees
    
    data = specs['data']
    long_f = specs['long_f']
    lat_f = specs['lat_f']
    cellsize = specs['cellsize']
    n_rows = specs['nrows']
    n_cols = specs['ncols']
    
    col = long_f(long)
    row = lat_f(lat)
    
    horz_pixel = deg_dist_at_lat(lat) * cellsize
    vert_pixel = deg_dist_at_lat(0) * cellsize
    horz_window = int(scan / horz_pixel)
    vert_window = int(scan / vert_pixel)
    row_ran = np.mod(np.arange(row-vert_window,row+vert_window+1), n_rows).astype(int)
    col_ran = np.mod(np.arange(col-horz_window,col+horz_window+1), n_cols).astype(int)
    assert len(row_ran) == 2*vert_window + 1
    assert len(col_ran) == 2*horz_window + 1
    tmp = data[row_ran, :][:, col_ran]
    #print(tmp.shape, row_ran, vert_window)
    #print(tmp.shape, col_ran, horz_window)

    shape = tmp.shape
    assert tmp.shape == (2*vert_window+1, 2*horz_window+1), f'({2*vert_window+1}, {2*horz_window+1} != {shape}'
    idx = np.unravel_index(np.argmax(tmp), tmp.shape)
    assert tmp[idx[0], idx[1]] == np.max(tmp)
    row_max = row - vert_window + idx[0]
    col_max = col - horz_window + idx[1]
    assert row_max in row_ran, f'{row_max} not in {row_ran}'
    assert col_max in col_ran, f'{col_max} not in {col_ran}'
    
    
    horz_pixel = deg_dist_at_lat(row2lat(row, specs)) * cellsize
    vert_pixel = deg_dist_at_lat(0) * cellsize
    horz_window_avg = int(avg / horz_pixel)
    vert_window_avg = int(avg / vert_pixel)
    row_ran = np.mod(np.arange(row_max-vert_window_avg,row_max+vert_window_avg+1), n_rows).astype(int)
    col_ran = np.mod(np.arange(col_max-horz_window_avg,col_max+horz_window_avg+1), n_cols).astype(int)
    tmp = data[row_ran, :][:, col_ran]
    tmp = tmp[tmp > 0]
    if len(tmp) == 0:
        density = 0
    else:
        density = np.mean(tmp)
   
    
    ret = {
        'vert_window': vert_window,
        'horz_window': horz_window, 
        'vert_window_avg': vert_window_avg,
        'horz_window_avg': horz_window_avg, 
        'row': row,
        'col': col,
        'row_max': row_max,
        'col_max': col_max,
        'row_top': row + vert_window,
        'row_bottom': row - vert_window,
        'col_left': col - horz_window,
        'col_right': col + horz_window,
        'horz_pixel': horz_pixel,
        'vert_pixel': vert_pixel,
        'density': density, 
        'idx': idx,
        'shape': shape,
    }
    pt = lambda x: [print(k, v) for k, v in x.items()]
    assert ret['row_max'] <= ret['row_top'] and ret['row_max'] >= ret['row_bottom'], pt(ret)
    assert ret['col_max'] <= ret['col_right'] and ret['col_max'] >= ret['col_left'], pt(ret)
    
    return ret

specs = get_coordinate_functions(specs)

airports = pd.read_csv('data/AirportInfo.csv')
g = lambda lat, long: augment(long=long, lat=lat, specs=specs)
airports = [{**row, **g(lat=row['Lat'], long=row['Lon'])} for _, row in airports.iterrows()]
airports = pd.DataFrame(airports)
airports.index = airports.NodeName.astype(str)
wuhan_factor =  4 / airports.loc['WUH', 'density'] # R_0 / density for Wuhan
airports['R0'] = airports.density * wuhan_factor
airports['p_outbreak'] = 1 - 1 / airports.R0
airports.loc[airports.p_outbreak < 0, 'p_outbreak'] = 0

# vis = airports.loc[airport_list]

# plt.close('all')
# fig, ax= plt.subplots(1,1, figsize=(30,20))
# dd = np.log(1+data)
# im = ax.imshow(dd, cmap=cm.gray)
# #fig.colorbar(im, ax=ax)
# ax.scatter(vis.col_left, vis.row, label='Window', color='b')
# ax.scatter(vis.col_right, vis.row, color='b')
# ax.scatter(vis.col, vis.row_top, color='b')
# ax.scatter(vis.col, vis.row_bottom, color='b')

# ax.scatter(vis.col, vis.row, label='Airports', color='r')
# ax.scatter(vis.col_max, vis.row_max, label='Maximizers', color='g')

# for _, row in vis.iterrows():
#     ax.annotate(row['NodeName'] + ' airport', (row['col']+1, row['row']), color='r', fontsize=15)
#     ax.annotate('Density= ' + str(int(row['density'])) + ' ' + row['NodeName'], (row['col_max']+1, row['row_max']), color='g', fontsize=15)
    
# plt.legend()
# plt.tight_layout()

In [48]:
filename = 'data/Prediction_Monthly.csv'
travel = pd.read_csv(filename)
nodes = airports.NodeName.values
travel = travel.query('Origin in @nodes and Dest in @nodes')
#travel['month_cnt'] = travel.groupby(['Origin', 'Dest']).Month.transform('count')
travel = travel.groupby(['Origin', 'Dest']).mean().reset_index(drop=False).drop('Month',axis=1)
airport_p_outbreak = dict(zip(airports.index, airports.p_outbreak))
travel = travel.assign(origin_p_outbreak=travel.Origin.map(airport_p_outbreak),
                       dest_p_outbreak=travel.Dest.map(airport_p_outbreak))
travel.dropna()
travel['outgoing_total'] = travel.groupby('Origin').Prediction.transform('sum')
travel = travel.query('outgoing_total > 0')

travel['P_ij'] = travel.Prediction / travel.outgoing_total
travel['risk_ij'] = travel.P_ij - travel.P_ij * travel.dest_p_outbreak
travel['risk_i'] = travel.groupby('Origin').risk_ij.transform('sum')
travel['origin_lon'] = travel.Origin.replace(airports.Lon)
travel['origin_lat'] = travel.Origin.replace(airports.Lat)
travel['dest_lon'] = travel.Dest.replace(airports.Lon)
travel['dest_lat'] = travel.Dest.replace(airports.Lat)
travel['red'] = travel.risk_i.clip(lower=0, upper=1)
travel['blue'] = (1 - travel.risk_i).clip(lower=0,upper=1)
travel['green'] = 0

assert np.max(np.abs(travel.groupby('Origin').P_ij.sum()-1)) < 1e-9


big = travel.sample(n=15000, weights='Prediction', axis=0)
small = travel.sample(n=500, weights='Prediction', axis=0)

travel.query('Prediction <= 0')

,Origin,Dest,Prediction,lower,upper,origin_p_outbreak,dest_p_outbreak,outgoing_total,P_ij,risk_ij,risk_i,origin_lon,origin_lat,dest_lon,dest_lat,red,blue,green
11667,IQQ,MIA,0.0,0.0,3.0,0.0,0.383,43466.083333,0.0,0.0,0.787379,-70.18,-20.55,-80.28,25.8,0.787379,0.212621,0
25029,VCP,MIA,0.0,0.0,1.0,0.0,0.383,98614.083333,0.0,0.0,0.621808,-47.14,-23.01,-80.28,25.8,0.621808,0.378192,0
25335,VLN,MIA,0.0,0.0,1.0,0.0,0.383,11415.333333,0.0,0.0,0.884813,-67.92,10.15,-80.28,25.8,0.884813,0.115187,0


In [49]:
plt.close('all')

ax = plt.axes(projection=ccrs.PlateCarree())
#ax.stock_img()

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
#ax.add_feature(cfeature.BORDERS, linestyle=':')
#ax.add_feature(cfeature.LAKES, alpha=0.5)
#ax.add_feature(cfeature.RIVERS)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)


plt.plot(big[['origin_lon', 'dest_lon']].T,
         big[['origin_lat', 'dest_lat']].T,
         alpha=0.05, 
         color='r',
         transform=ccrs.Geodetic())
plt.tight_layout()
plt.show()

In [50]:
plt.close('all')

ax = plt.axes(projection=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)
plt.scatter(travel.origin_lon, travel.origin_lat, color = travel[['red', 'green', 'blue']].values)

plt.tight_layout()
plt.show()

In [59]:
wuhan = travel.query('Origin == "WUH"')
wuhan.sort_values('risk_ij', inplace=True)
wuhan = wuhan.assign(destination=wuhan.Dest.replace(airports.OAGName))
wuhan.destination

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


25710                           Yantai China
25689         Shanghai (Hongqiao Intl) China
25708                        Yun Cheng China
25691                     Shijiazhuang China
25704                         Xiangfan China
25675      Seoul(Incheon Intl) Rep. of Korea
25698       Taipei(Sung Shan) Chinese Taipei
25681                            Linyi China
25682                      Macau Macao China
25692                          Shantou China
25697    Taipei(Taoyuan Intl) Chinese Taipei
25701                           Urumqi China
25686                          Nantong China
25707                           Xining China
25674                         Huangyan China
25711                           Zhuhai China
25672                  Hong Kong(Intl) China
25663                        Changchun China
25700                          Taiyuan China
25703                             Wuxi China
25684                          Nanjing China
25695                          Qingdao China
25676     

In [60]:
airports.loc["NRT"]

NodeName                           NRT
OAGName            Tokyo(Narita) Japan
Name                    Narita Airport
City                             Tokyo
Lon                             140.39
Lat                              35.77
vert_window                          6
horz_window                          7
vert_window_avg                      3
horz_window_avg                      3
row                               1301
col                               7689
row_max                           1302
col_max                           7682
row_top                           1307
row_bottom                        1295
col_left                          7682
col_right                         7696
horz_pixel                     3.75815
vert_pixel                     4.63312
density                        3564.77
idx                             (7, 0)
shape                         (13, 15)
R0                             1.50453
p_outbreak                    0.335338
Name: NRT, dtype: object